# TP Transcriptomique 1 - TP5 - 2020-2021


## Analyse de données de puces (MicroArrays)

### Adaptation du TP MeV Testis Vs Ovary, sous forme de Jupyter Notebook en R

Selon les explications du site https://wiki.bits.vib.be/index.php/Analyze_your_own_microarray_data_in_R/Bioconductor

---
**Avant toute chose**

<mark>Avant de démarrer cette analyse, sauvegardez une copie de ce notebook pour backup. <mark>

Vous pouvez aussi faire des copies de ce notebook au fur et à mesure de la progression

---

__*Rappels sur l'utilisation des notebooks*__

- Pour ajouter une nouvelle cellule, cliquez sur l'icône "+" dans la barre des menus *
- Vous pouvez "cliquer-glisser" pour bouger une cellule *
- Vous pouvez choisir le type de cellule dans le petit menu déroulant dans la barre des menus : *
    - 'Code' pour entrer des lignes de commandes à executer *
    - 'Markdown cells' pour ajouter simplement du texte, formatable avec quelques signes *
- Pour executer une cellule 'Code', pressez SHIFT+ENTER ou cliquez sur l'icône "play" *
- Pour afficher une cellule 'Markdown', pressez SHIFT+ENTER ou cliquez sur l'icône "play" *
- Pour modifier une cellule 'Markdown', double-cliquez dessus*

---

## I - Préparation de l'espace de travail

Ceci est un jupyter notebook en R, ce qui signifie que les commandes seront des commandes R directement interprétables par Plasma. 
Comme pour tout projet en R, nous devons d'abord choisir le répertoire de travail. Par défaut, ce sera celui de l'environnement déployé lors de l'ouverture du TP.
Puis nous devons indiquer à R les paquets à charger qui seront nécessaires pour l'analyse. 


In [ ]:
# Le répertoire de travail est d'office attribué au répertoire de l'environnement par Jupyter Hub
# setwd("~/m1meg-ue1-tp5-r/")

In [ ]:
# Chargement des packages dans l'environnement présent
library(Matrix)
library(lattice)
library(fdrtool)
library(rpart)
library(Biobase)
library(Biostrings)
library(mouse4302cdf)
library(ggplot2)
library(limma)
library(affy)
library(affyPLM)
library(simpleaffy)
library(genefilter)

Nous pouvons vérifier ces éléments avec les commandes suivantes : 

In [ ]:
getwd()
sessionInfo()

## II - Analyse des puces 

Nous allons analyser un ensemble de données d’expression, obtenues sur puces d’expression de type Affymetrix. Il s’agit d’une portion de la série **GSE12989**, accessible par une des deux bases de données d’expression génique (“repository”) : Gene Expression Omnibus (GEO) http://www.ncbi.nlm.nih.gov/geo/
Sur GEO, un ensemble d'échantillons composent une **Série**, avec un identifiant commençant par GSE.

Ouvrez le lien https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE12989

La description de la série indique :

Status 			Public on Jun 24, 2009
Title 			Foxl2 functions throughout mouse ovary development
Organism 		Mus musculus
Experiment type 	Expression profiling by array
Summary 		This SuperSeries is composed of the following subset Series:
- GSE12905: Foxl2 functions in sex determination and histogenesis throughout mouse ovary development, analyzed by Affymetrix arrays
- GSE12942: Foxl2 functions in sex determination and histogenesis throughout mouse ovary development, analyzed by Agilent arrays
Citation(s) 
•	Garcia-Ortiz JE, Pelosi E, Omari S, Nedorezov T et al. Foxl2 functions in sex determination and histogenesis throughout mouse ovary development. BMC Dev Biol 2009 Jun 18;9:36. PMID: 19538736

Les puces Affymetrix utilisées dans le subset GSE12905 suivent la plateforme (design de la puce) GPL1261, c’est à dire le Affymetrix Mouse Genome 430 2.0 Array (nom commercial de la puce).


> <mark>**Question 1 :**<mark>  
> Combien d’échantillons analysés avec cette puce sont présents dans GEO ? 

*Double-cliquez ici pour entrer votre réponse ou des notes*

### 1 - Récupération des données

Les fichiers CEL qui nous intéressent ont déjà été télécharchés sur Plasmabio. Il suffit donc d'indiquer à R où aller les chercher, et stocker cette information dans la variable *celpath*


In [ ]:
# Chemin d'accès aux données sur plasmabio :
celpath = "/srv/data/meg-m1-ue1/DataTP5/"

Nous allons maintenant charger les données de ces fichiers dans un objet R approprié : AffyBatch

In [ ]:
# Import des données d'intensité de fluorescence brutes des sondes depuis les fichiers CEL, 
# et ajout dans un objet R AffyBatch
data = ReadAffy(celfile.path=celpath)

Cet objet AffyBatch est accessible via des méthodes contenues dans le package affy.

In [ ]:
# Récupération de l'annotation des échantillons
ph = data@phenoData
ph
ph@data

> <mark>**Question 2 :**<mark>  
> Combien d’échantillons de testicules et d’ovaires sont présents ? 
Combien y a t-il de réplicats par stade de développement ? 

*Double-cliquez ici pour entrer votre réponse ou des notes. Par la suite, il vous suffira de sélectionner la cellule de la question, puis d'ajouter une cellule markdown avec le "+" pour entrez vos réponses* 

In [ ]:
#Récupération de l'annotation des sondes
feat = data@featureData
feat
feat@data

In [ ]:
# Récupération du nom du fichier CDF associé aux puces
cdfName(data)

In [ ]:
# Récupération du nombre de probe sets représentés sur les puces
# (La fonction length() compte le nombre d'items dans un vecteur)
length(featureNames(data))

> <mark>**Question 3 :**<mark>  
> Combien de probes sets sont analysés grâce à cette puce ? 


### 2 - Contrôle qualité des données de puces

Nous commençons par attribuerdes noms un peu plus gérables aux échantillons :


In [ ]:
# Attribution de noms informatifs aux échantillons
ph@data[ ,1] = c("Testis_E13_1","Testis_E13_2","Testis_E13_3","Testis_P0_1","Testis_P0_2","Testis_P0_3","Ovary_E13_1","Ovary_E13_2","Ovary_E13_3", "Ovary_P0_1", "Ovary_P0_2", "Ovary_P0_3")
ph@data

#### Histogrammes de distribution des données 
Puis nous créons 2 histogrammes de la distribution des données (avec ggplot2).
Le premier affiche la répartition des données brutes, le second celle des valeurs passées en log2.

In [ ]:
# Récupérations des intensités PM (sondes Perfect Match dans les probe sets)
pmexp = pm(data)

# La méthode dim() fournit le nombre d'éléments dans la matrice, 
# c'est à dire le nomdre de sondes (lignes) et le nombre d'échantillons (colonnes)
dim(pmexp)

On crée des vecteurs qui vont nous servir comme colonnes de la dataframe pour les histogrammes
  - un pour y mettre les noms des échantillons, appelé **sampleNames**
  - un pour y stocker les intensités de fluorescences brutes, applelé **nologs**
  - un pour y stocker les intensités passées en log2, appelé **logs**

In [ ]:
# initialisation de 3 vecteurs
sampleNames = vector()
nologs = vector()
logs = vector()

# remplissage des vecteurs avec les données
for (i in 1:12)
{
  sampleNames = c(sampleNames,rep(ph@data[i,1],dim(pmexp)[1]))
  nologs = c(nologs, pmexp[,i])
  logs = c(logs,log2(pmexp[,i]))
}

# Vérification avec les premières valeurs
nologs[1:10]
logs[1:10]

In [ ]:
# Après avoir rempli les vecteurs, nous combinons sampleNames et nologs ou logs dans 2 dataframes
nologData = data.frame(nologInt=nologs,sampleName=sampleNames)
logData = data.frame(logInt=logs,sampleName=sampleNames)

# Vérification avec les premières valeurs
nologData[1:5,]
logData[1:5,]

dim(nologData)
dim(logData)


In [ ]:
# Maintenant nous pouvons créer les 2 histogrammes: 
dataHist1 = ggplot(nologData, aes(nologInt, colour = sampleName)) 
dataHist1 + geom_density()

In [ ]:
# Histogramme avec les valeurs en log2
dataHist2 = ggplot(logData, aes(logInt, colour = sampleName)) 
dataHist2 + geom_density()

> <mark>**Question 4 :**<mark>  
> Que pouvez-vous dire de la distribution des données ? Vous semble t-elle logique pour des transcriptomes ?

#### Boxplots de distribution des données 
Nous utilisons les mêmes dataframes pour créer 2 boxplots, 
de façon à comparer la répartition des données entre échantillons


In [ ]:
dataBox1 = ggplot(nologData,aes(sampleName,nologInt))
dataBox1 + geom_boxplot()

In [ ]:
dataBox2 = ggplot(logData,aes(sampleName,logInt))
dataBox2 + geom_boxplot()

> <mark>**Question 5 :**<mark>  
> Pouvez-vous déterminer si les données ont été normalisées entre puces ou pas ? 

#### MA plots
Les plots MA permettent de visualiser la variablité de l'expression des gènes, et sa répartition en fonction du niveau d'expression.
Ici, chaque puce est compré à une pseudo-puce, pour laquelle chaque sonde est associée à la valeur médiane de son intensité dans tous les échantillons.

Dans un plot MA, M est figuré en fonction de A :

    - M est la différence d'intensité d'une sonde sur la puce testée et l'intensité de cette même sonde sur la pseudo-puce
    M = logPMInt_array - logPMInt_pseudoarray
    - A est la moyenne des intensités de cette sonde sur la puce testée et sur la pseudo-puce
    A = (logPMInt_array + logPMInt_pseudoarray)/2   
    
(les plots se créent dans des fichier séparés, colonne de gauche)

In [ ]:
# Création des MA plots pour chaque échantillon.
for (i in 1:12)
{
  name = paste("MAplot",i,".jpg",sep="")
  jpeg(name)
  MAplot(data,which=i)
  dev.off()
}


Ouvrez les MA plots et examinez-les.

> <mark>**Question 6 :**<mark>  
> Que pouvez-vous constater ? Pensez-vous qu'il faille normaliser les donnéees ?

### 3 - Normalisation des données
Méthode GCRMA
The standard method for normalization is RMA. RMA is one of the few normalization methods that only uses the PM probes:
   - Background correction to correct for spatial variation within individual arrays: a background-corrected intensity is calculated for each PM probe in such a way that all background corrected intensities are positive
   - Log transformation to improve the distribution of the data: the base-2 logarithm of the background corrected intensity is calculated for each probe. 
     The log transformation will make the data less skewed and more normally distributed and provide an equal spread of up- and downregulated expression ratios
   - Quantile normalization to correct for variation between the arrays: equalizes the data distributions of the arrays and make the samples completely comparable
   - Probe normalization to correct for variation within probe sets: equalizes the behavior of the probes between the arrays and combines normalized data values of probes from a probe set into a single value for the whole probe set

GCRMA is based on RMA, having all the good sides of RMA. 
The difference lies in the background correction, all other steps are the same. 
GCRMA corrects for non-specific binding to the probes in contrast to RMA which completely ignores the issue of non-specific binding.
GCRMA uses probe sequence information (hence "GC")to estimate probe affinity to non-specific binding. 

*Normalement, nous utiliserions la méthode gcrma directement dans R, grâce à cette commande : 
library("gcrma")
data.gcrma = gcrma(data)*

*Pour des raisons techniques en cours de résolution, cela ne marche pas actuellement sur Plasma.*

*Nous allons donc charger directement le résultat de la normalisation pour la suite de l'analyse*


In [ ]:
# Chargement de la library, nécessaire pour la suite
# library("gcrma")

# Chargement de l'objet "data_gcrma.RData", pour le réintégrer dans la procédure d'analyse
load("data_gcrma.RData")

Nous allons maintenant vérifier l'effet de la normalisation 

In [ ]:
# Box plots des valeurs normalisées
# création d'une data frame avec seulement les valeurs d'expression normalisées
normexpr.gcrma = data.frame(exprs(data.gcrma))
normexpr.gcrma[1:5,]


# initialisation des vecteurs
sampleNames = vector()
normlogs = vector()

# remplissage des vecteurs
for (i in 1:12)
{
    sampleNames = c(sampleNames,rep(ph@data[i,1],dim(data.gcrma)[1]))
    normlogs = c(normlogs,normexpr.gcrma[,i])
}

normlogs[1:10]

# création de la dataframe avec les 2 vecteurs
normData = data.frame(norm_logInt=normlogs,sampleName=sampleNames)
normData[1:5,]

# création des graphs
dataBox3 = ggplot(normData, aes(sampleName,norm_logInt))
dataBox3 + geom_boxplot() + ylim(2,16) + ggtitle("after normalization")


> <mark>**Question 7 :**<mark>
Que pouvez-vous constater ? Pensez-vous que la normalisation entre échantillons a fonctionné ?

Pour voir ce que la normalisation a fait aux données de chaque échantillon, nous allons refaire les MA plots.

In [ ]:
# MAplots des données normalisées 

for (i in 1:12)
{
  name = paste("MAplot-normData",i,".jpg",sep="")
  jpeg(name)
  MAplot(data.gcrma,which=i)
  dev.off()
}


> <mark>**Question 8 :**<mark>
Qu'en pensez-vous ?

### 4 - Groupement des échantillons : PCA

Pour voir si les échantillons se regroupent de façon cohérente, nous allons tenter une analyse PCA (Principal omponant Analysis).  
Cette analyse vous sera expliquée plus en détail lors du TP PCA de l'UE5 (avec Fabien Fauchereau)


In [ ]:
# PCA plot

color=c('green','green','green','blue','blue','blue', 'red','red','red', 'orange', 'orange', 'orange' )
data.PCA = prcomp(t(normexpr.gcrma),scale.=FALSE)
plot(data.PCA$x[1:12],col=color)


> <mark>**Question 8 :**<mark>  
>Que pensez-vous de la répartition des réplicats ?


### 5 - Identification des gènes DE (Différentiellement Exprimés)

Nous allons maintenant identifier les gènes dont l'expression varie entre les conditions.   
Nous allons d'abord effectuer une comparaison simple entre 2 groupes, ici le sexe gonadique.  
Nous attribuons à chaque échantillon sa valeur selon ce critère ("Testis" ou "Ovary"), puis nous indiquons à R que c'est cet élément qui va nous servir de facteur de comparaison.  
Puis une matrice de contraste est créée pour comparer les 2 groupes.


In [ ]:
# Comparaison simple entre 2 groupes

# Ajout d'une colonne pour caractériser les échantillons
ph@data[ ,2] = c("Testis","Testis","Testis","Testis","Testis","Testis", "Ovary", "Ovary", "Ovary", "Ovary", "Ovary", "Ovary")
colnames(ph@data)[2]="SexGonad"

# On vérifie que le facteur a été correctement attribué
ph@data

# On forme les 2 groupes
groups = ph@data$SexGonad

f = factor(groups,levels=c("Testis","Ovary"))

design = model.matrix(~0 + f)
colnames(design) = c("Testis","Ovary")

data.fit = lmFit(data.gcrma,design)

# montre les 10 premières valeurs dans les 2 groupes
data.fit$coefficients[1:10,]

contrast.matrix = makeContrasts(Ovary-Testis,levels=design)
data.fit.con = contrasts.fit(data.fit,contrast.matrix)
data.fit.eb = eBayes(data.fit.con)


On affiche ensuite les résultats pour les 10 gènes les plus significativement différents :

In [ ]:
# indique le noms des colonnes dans la matrice de résultats
names(data.fit.eb)

# montre les p-values associées au test pour les 10 premiers gènes
data.fit.eb$p.value[1:10,]

On peut visualiser les p-values obtenues avec un **Volcano Plot**  
(se crée aussi dans un fichier séparé, colonne de gauche)

In [ ]:

# Volcano plot
name = "Volcano1.jpg"
jpeg(name)
volcanoplot(data.fit.eb,coef=1,highlight=50, style = "p-value", xlab = "log2 Fold Change", ylab = "-log10 p", names = NULL, hl.col = "blue")
dev.off()


><mark>**Question 9 :**<mark>  
>    Pensez-vous que les p-values soient interprétables directement ? Quelle étape importante doit d'abord être effectuée ?

### Correction des p-values pour les tests multiples
The topTable() method returns a table ranking the genes according to evidence for differential expression.  
Additionally, the topTable() method will adjust the p-value obtained from the moderated t-test for multiple testing. 
The adjustment method is defined by the adjust.method argument. In this case, the adjustment is done using BH which is Benjamini and Hochberg's method to control the FDR.


In [ ]:
options(digits=2)
tab = topTable(data.fit.eb,coef=1,number=5000,adjust.method="BH")

tab[1:10,]

# la colonne B indique le log odds scores, 
# c'est à dire le logarithme de la vraisemblance de la DE par rapport à l'hypothèse nulle
# (ratio des vraisemblances, transformée en log).
# Souvenez-vous des cours et TP sur la méthode des lod-scores en génétique humaine de L3 !)


On peut se servir de la même fonction topTable() pour sélectionner les gènes selon des seuils, par exemple de p-value

In [ ]:
# Sélection des gènes selon la adjusted p-value, sous un certain seuil

topgenesAdjP = tab[tab[, "adj.P.Val"] < 0.01, ]

# nombre de gènes correpsondant au critère :
dim(topgenesAdjP)[1]

topgenesAdjP[1:10,]



><mark>**Question 9 :**<mark>  
>    Indiquez le nombre de gènes qui sont en dessous des seuils de p-value suivants :  
>     adj p < 0.05  
>     adj p < 0.01  
>     adj p < 0.001  

In [ ]:
# pour séparer les gènes up- and down-regulated, on inclut un seuil sur le log fold change :
  
topUpAdjP = topgenesAdjP[topgenesAdjP[, "logFC"] > 1, ]
dim(topUpAdjP)[1]
topDownAdjP = topgenesAdjP[topgenesAdjP[, "logFC"] < -1, ]
dim(topDownAdjP)[1]


><mark>**Question 10 :**<mark>  
>    Pour chacun des seuilsde p-value précédents, indiquez le nombre de gènes up et down-régulés

><mark>**Question 11 :**<mark>  
>    Est-ce que l'identification de ces gènes DE nous permet de capturer la totalité des variations des transcriptomes dans ces échantillons ?  
> Quel autre facteur n'a pas été pris en compte ?  
> Pensez-vous qu'une analyse entre 2 groupes en factorisant par ce 2ème facteur suffise ?

### Analyse ANOVA
En fait nous avons réellement 4 groupes, et il vaut mieux utiliser un test statistique plus fin pour bien voir les différences entre ces 4 groupes.
Une ANOVA à 2 facteurs est le bon choix ici. Cette méthode permet de tester l'effet de chaque facteur ainsi que de leur éventuelle interaction.  
Nous commençons par redéfinir des groupes, mais ce coup-ci, en établissant 2 facteurs   
 - fs : facteur Sexe Gonadique
 - fd : facteur de stades de développement  
Chacun des 2 facteurs possèdent 2 niveaux (on pourrait donc en spécifier plus si nécessaire, par exemple pour les stades de développement).  
On effectue ensuite l'analyse en disant que l'on veut l'effet des 2 facteurs et leur interaction (sigen * dans la fonction model.matrix().  



In [ ]:
ph@data[ ,3] = c("Testis","Testis","Testis","Testis","Testis","Testis", "Ovary", "Ovary", "Ovary", "Ovary", "Ovary", "Ovary")
colnames(ph@data)[3]="GonadSex"
ph@data[ ,4] = c("E13","E13","E13","P0","P0","P0", "E13", "E13", "E13", "P0", "P0", "P0")
colnames(ph@data)[4]="DevStage"
ph@data

groupsS = ph@data$GonadSex 
groupsD = ph@data$DevStage
fs = factor(groupsS,levels=c("Testis","Ovary"))
fd = factor(groupsD,levels=c("E13","P0"))

factored.design = model.matrix(~fs*fd)
data.fit4 = lmFit(data.gcrma,factored.design)

# Affichage des résultats :
# data.fit4[1:10,]
# Lmfit() will fit a linear model to the data. It will create a data frame called data.fit4 containing 4 columns:
#   the first column contains the intercept of the linear model: it's the mean log expression in the testis samples
#   the second column compares Ovary samples with Testis
#   the third column compares P0 with E13
#   the fourth column contains the interaction showing you if the genes that respond to the gonadal sex are influenced by the developmental stage

# Performing the two-factor ANOVA is now done using eBayes(). 
data.fit.eb4 = eBayes(data.fit4)

# Affichage des p-value pour chacun des facteurs et leur interactions, pour les 15 premiers gènes
data.fit.eb4$p.value[1:15,1:4]


La correction des tests multiples s'effectue ici avec une autre fonction : decideTests()  
On peut là aussi spécifier des seuils pour ne garder que les gènes à considérer comme DE.  
La fonction retourne alors des valeurs 0, 1, -1 selon que le gène remplit les critères de sélection.

In [ ]:
# Correction des tests multiples avec une comparaison multiple (i.e une 2-way ANOVA)
# The decideTests() method will perform multiple testing adjustment on these p-values. 
# Additionally, it will evaluate for each gene whether the results data.fit.eb fulfill the criteria for differential expression that you specify.
# For each gene and each comparison it will generate the following output:
#   -1: significantly downregulated
#    0: no significant evidence of differential expression
#    1: significantly upregulated
# The p.value argument specifies the FDR and the lfc argument specifies the minimal fold change that is required to be considered DE.

DEresults = decideTests(data.fit.eb4,method='global',adjust.method="BH",p.value=0.05,lfc=1)
DEresults[1:10,]

DEresultsStrict = decideTests(data.fit.eb4,method='global',adjust.method="BH",p.value=0.01,lfc=2)
DEresultsStrict[1:10,]

### Diagramme de Venn

Pour terminer, on peut afficher des diagrammes de Venn qui montrent comment se répartissent les gènes DE selon les facteurs considérés.

In [ ]:
# Diagramme de Venn des gènes DE
# "both" for all differentially expressed genes, "up" for up-regulated genes only or "down" for down-regulated genes only.

vennDiagram(DEresults[,2:4], include="down", names= c("Gonad Sex", "Dev Stage", "Interaction"), counts.col=c("black"), circle.col = c("red", "blue", "green3"))
vennDiagram(DEresultsStrict[,2:4], include="up", names= c("Gonad Sex", "Dev Stage", "Interaction"), counts.col="red", circle.col = c("red", "blue", "green3"))



><mark>**Question 12 :**<mark>   
> <mark>   Reprenez et modifiez le code dess cellules précedentes de façon à produire un diagramme de Venn :  <mark>  
> <mark>  - sur les gènes avec une adj pvalue < 0.01  <mark>  
> <mark>  - et avec un log fold change supérieur à 1.5      <mark>                                     
> <mark>  - les gènes up et down-régulés   <mark>  
> <mark>  - avec l'écriture des comptages en rouge  <mark> 
  

In [ ]:
# rentrez ici le code modifié pour répondre à la question 12



><mark>**Question 13 :**<mark>   
> Combien de gènes sont DE à la fois selon le sexe et le stade de développement ?  
> Combien de gènes ont leur expression modifiée selon le stade de développemnt, mais différemment selon le sexe ?